<a href="https://colab.research.google.com/github/ronyates47/Gedcom-Utils/blob/main/A_v_18_1319_Yates_Primary_2023_gedcomEXP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import csv
import glob
from gedcom.element.individual import IndividualElement
from gedcom.parser import Parser
import pandas as pd

class GedcomDataset:
    def __init__(self, gen_person):
        self.gen_person = gen_person
        self.extractable_detail = {}
        self.anchor_gen1 = None  # Initialize anchor_gen1 here

    def add_extractable_detail(self, key, value):
        self.extractable_detail[key] = value

    def get_gen_person(self):
        name = self.extractable_detail.get('NAME', '')
        parts = name.split('/', 1)
        first_name = parts[0].split(' ')[0]
        last_name = parts[1].rstrip('/') if len(parts) > 1 else ""
        self.anchor_gen1 = last_name.replace(" ", "") + first_name.replace(" ", "")
        return self.gen_person.strip('@')

    def get_anchor_gen1(self):
        return self.anchor_gen1

    def get_extractable_NPFX(self):
        return self.extractable_detail.get('NPFX', '')

    def get_extractable_cm(self):
        npfx_value = self.extractable_detail.get('NPFX', '')
        if '&' in npfx_value:
            cm_value = npfx_value.split('&')[0].strip()
        else:
            cm_value = npfx_value.strip()
        try:
            int(cm_value)
            return cm_value
        except ValueError:
            return 'error'

    def get_extractable_sort(self):
        npfx_value = self.extractable_detail.get('NPFX', '')
        if '&' in npfx_value:
            sort_value = npfx_value.split('&')[1].strip()
            return sort_value
        else:
            return ''

    def get_extractable_FAMC(self):
        return self.extractable_detail.get('FAMC', '').strip('@')

# Function definitions
def extract_id(record):
    id_start = record.find('@') + 1
    id_end = record.find('@', id_start)
    return record[id_start:id_end]

def extract_name(record):
    name_start = record.find('1 NAME ') + 6
    name_end = record.find('\n', name_start)
    name = record[name_start:name_end]
    first_name, last_name = name.split('/', 1)
    first_name = first_name[:10] # Use slicing syntax to extract the first 10 characters of the first_name variable
    last_name = last_name[:10].rstrip('/') # Use slicing syntax to extract the first 10 characters of the last_name variable
    return last_name.replace(" ", "") + first_name.replace(" ", "")

# Global dictionary to hold name to ID mapping
name_to_id = {}

class Gedcom:
    def __init__(self, file_name):
        self.file_name = file_name
        self.gedcom_datasets = []
        self.filter_pool = []

    @staticmethod
    def get_standard_name(file_path):
        file_name = file_path.split('/')[-1]
        if '.' in file_name:
            file_name = file_name.rsplit('.', 1)[0]
        standard_name = file_name.replace(' ', '_').lower()
        return standard_name

class Gedcom:
    def __init__(self, file_name):
        self.file_name = file_name
        self.gedcom_datasets = []
        self.filter_pool = []

    def parse_gedcom(self):
        global name_to_id  # Declare name_to_id as global to modify it
        with open(self.file_name, 'r', encoding='utf-8-sig') as f:
            gedcom_lines = f.readlines()
        current_dataset = None
        npfx_count = 0
        total_count = 0

        for line in gedcom_lines:
            parts = line.strip().split(' ', 2)
            level = int(parts[0])
            tag = parts[1]
            value = parts[2] if len(parts) > 2 else None

            if level == 0 and tag.startswith('@') and tag.endswith('@') and value == 'INDI':
                total_count += 1
                current_dataset = GedcomDataset(tag)
                self.gedcom_datasets.append(current_dataset)

                # Populate name_to_id
                individual_name = current_dataset.get_anchor_gen1()
                individual_id = current_dataset.get_gen_person()
                name_to_id[individual_name] = individual_id

            elif current_dataset is not None:
                if level == 1 and tag in ['NAME', 'FAMC']:
                    current_key = tag
                    current_dataset.add_extractable_detail(current_key, value)

                elif level == 2 and tag == 'NPFX':
                    npfx_count += 1
                    current_dataset.add_extractable_detail(tag, value)

        print(f'GEDCOM contained {total_count} total records')
        print(f'Records tagged and filtered by NPFX: {npfx_count}')

        # First level of filtering: Filter those with NPFX
        for dataset in self.gedcom_datasets:
            if dataset.get_extractable_NPFX():
                self.filter_pool.append(dataset)

        # Check if manual filtering should be applied
        manual_filter_activated = True  # or False depending on your situation

        # Second level of filtering: Apply manual filter from Excel sheet
        if manual_filter_activated:
            import pandas as pd  # Assuming you haven't imported it yet
            try:
                df = pd.read_excel('filtered_ids.xlsx')
            except FileNotFoundError:
                print("filtered_ids.xlsx not found. Skipping second-level manual filter.")
            else:
                manual_filtered_ids = set(df['ID'])
                print(f"Manual filter IDs loaded: {len(manual_filtered_ids) - 1}")

                self.filter_pool = [dataset for dataset in self.filter_pool if dataset.get_gen_person() in manual_filtered_ids]
                print(f"After manual filter, total records: {len(self.filter_pool)}")

def input_prime_surname(last_prime_surname=None):
    if last_prime_surname:
        last_name = input(f"Enter prime_surname (default: {last_prime_surname}): ")
        if not last_name:
            last_name = last_prime_surname
    else:
        last_name = input("Enter prime_surname: ")
    return last_name

def select_gedcom_file():
    gedcom_files = glob.glob('*.ged')
    if not gedcom_files:
        print("No GEDCOM files found.")
        return None

    print("Automatically selecting the first GEDCOM file.")
    return gedcom_files[0]

    while True:
        try:
            selected_num = int(input("Enter the number of the GEDCOM file you want to use: "))
            if 1 <= selected_num <= len(gedcom_files):
                return gedcom_files[selected_num - 1]
            else:
                print("Invalid number. Please enter a valid number from the list.")
        except ValueError:
            print("Invalid input. Please enter a valid number.")

gedcom_file_path = select_gedcom_file() # Call the function to let the user select the GEDCOM file
if gedcom_file_path:
    # Use the selected GEDCOM file path to create an instance of the Gedcom class
    gedcom_instance = Gedcom(gedcom_file_path)
    gedcom_instance.parse_gedcom()

    individuals = []  # Initialize the list of individuals

    for dataset in gedcom_instance.filter_pool:    # Iterate over the filter_pool list,add each last name and ID to list
        individual_id = dataset.get_gen_person()
        last_name = dataset.get_anchor_gen1()
        individuals.append((last_name, individual_id))

#    print(f'Records tagged and filtered by NPFX: {len(individuals)}')

    with open(gedcom_file_path, 'r') as file:    # Read the GEDCOM file and split it into individual and family records
        data = file.read()
    data = data.split('\n0 ')
    records = {extract_id(record): record for record in data}

def has_both_parents(records, mother_id, father_id):
    return mother_id in records and father_id in records

visited_pairs = set()
generation_table = []

def find_parents(individual_id, generation, records):
    # ... (same code, but store individual IDs in visited_pairs and generation_table)
    if individual_id not in records:
        return
    record = records[individual_id]
    famc_start = record.find('1 FAMC @') + 8
    famc_end = record.find('@', famc_start)
    famc_id = record[famc_start:famc_end]
    if famc_id not in records:
        return

    fam_record = records[famc_id]
    wife_start = fam_record.find('1 WIFE @') + 8
    wife_end = fam_record.find('@', wife_start)
    mother_id = fam_record[wife_start:wife_end]

    husb_start = fam_record.find('1 HUSB @') + 8
    husb_end = fam_record.find('@', husb_start)
    father_id = fam_record[husb_start:husb_end]

    if mother_id and mother_id in records and father_id and father_id in records:
        parent_pair = (father_id, mother_id)
        if parent_pair not in visited_pairs:
            visited_pairs.add(parent_pair)
            generation_table.append((generation, parent_pair))

#            print(f"Added to generation_table: {parent_pair} at generation {generation}")

    if mother_id:
        find_parents(mother_id, generation + 1, records)

    if father_id:
        find_parents(father_id, generation + 1, records)

#    print(f"Visited pairs so far: {visited_pairs}")


def extract_name(record):
    name_start = record.find('1 NAME ') + 6
    name_end = record.find('\n', name_start)
    name = record[name_start:name_end]
    first_name, last_name = name.split('/', 1)
    first_name = first_name[:10]
    last_name = last_name[:10].rstrip('/')
    return last_name.replace(" ", "") + first_name.replace(" ", "")

def find_distant_ancestors(individual_id, records, path=None):
    path = path if path is not None else []
    if path is None:
        path = [individual_id]
    else:
        path.append(individual_id)

    if individual_id not in records:
        return []

    record = records[individual_id]
    famc_start = record.find('1 FAMC @') + 8
    famc_end = record.find('@', famc_start)
    famc_id = record[famc_start:famc_end]

    if famc_id not in records:
        return [path]

    fam_record = records[famc_id]
    wife_start = fam_record.find('1 WIFE @') + 8
    wife_end = fam_record.find('@', wife_start)
    mother_id = fam_record[wife_start:wife_end]

    husb_start = fam_record.find('1 HUSB @') + 8
    husb_end = fam_record.find('@', husb_start)
    father_id = fam_record[husb_start:husb_end]

    if father_id is None and mother_id is None:
        return [path]

    paths = []
    if father_id:
        new_path = list(path)
        paths.extend(find_distant_ancestors(father_id, records, new_path))

    if mother_id:
        new_path = list(path)
        paths.extend(find_distant_ancestors(mother_id, records, new_path))

#    print(f"Distant ancestors paths for {individual_id}: {paths}")

    return paths
filtered_datasets = gedcom_instance.filter_pool

# Additional Function to isolate score calculation logic
def calculate_score(distant_ancestors_paths, records):
    # ... (same code, but works with individual IDs)
    name_paths = []
    for path in distant_ancestors_paths:
        name_path = [extract_name(records.get(id, '')) for id in path]
        name_paths.append(name_path)

    path_scores = {}
    for idx, name_path in enumerate(name_paths):
        score = 0
        for generation, name in enumerate(name_path):
            if 'Yates' in name:
                score += 1 * (generation + 1)
        path_scores[idx] = score

    if path_scores:
        winning_path_index = max(path_scores, key=path_scores.get)
        winning_path_score = path_scores[winning_path_index]
        winning_path_names = name_paths[winning_path_index]
        winning_path_ids = distant_ancestors_paths[winning_path_index]  # Get the IDs of the winning path
        print(f"314-Debug - winning_path_ids: {winning_path_ids}")  # Print the IDs of the winning path
    else:
        winning_path_index = None
        winning_path_score = 0
        winning_path_names = []
        print("Debug - winning_path_ids: None")  # Print None if no winning path found

    return winning_path_score, winning_path_names


# New function to filter the full ancestral line based on the DNA line
def filter_ancestral_line(dna_line_ids, full_ancestral_line_ids):
    return [pair for pair in full_ancestral_line_ids if any(id in pair for id in dna_line_ids)]

# Main Loop
for dataset in filtered_datasets:
    individual_id = dataset.get_gen_person()

    # Reset global variables for each new individual
    visited_pairs = set()
    generation_table = []

    # Find full ancestral line and distant ancestors
    find_parents(individual_id, 1, records)
    distant_ancestors_paths = find_distant_ancestors(individual_id, records)

    # Calculate score and get DNA line
    winning_path_score, winning_dna_line_ids = calculate_score(distant_ancestors_paths, records)

    # Filter the full ancestral line based on the DNA line
    filtered_ancestral_line_ids = filter_ancestral_line(winning_dna_line_ids, [pair for generation, pair in generation_table])

# Main Loop
# Assuming name_to_id is populated elsewhere in your code
name_to_id = {}  # Placeholder, populate this after reading the GEDCOM file

def process_individual(individual_id, gedcom_instance, records):
    global generation_table
    global visited_pairs

    # Resetting the variables for each individual
    generation_table = []
    visited_pairs = set()

    # Finding parents and distant ancestors
    find_parents(individual_id, 1, records)
    distant_ancestors_paths = find_distant_ancestors(individual_id, records)

    # Calculate the winning path
    winning_path_score, winning_path_names = calculate_score(distant_ancestors_paths, records)

    # Extract other information for this individual
    individual_data = {}
    for dataset in gedcom_instance.filter_pool:
        if dataset.get_gen_person() == individual_id:
            individual_data['cM'] = dataset.get_extractable_cm()
            individual_data['Sort'] = dataset.get_extractable_sort()
            break

    # Filter the parent pairs based on the winning path
    selected_dna_branch_ids = {name_to_id[name] for name in winning_path_names if name in name_to_id}
    filtered_parent_pairs = {}
    for generation, parent_pair_tuple in generation_table:
        parent1, parent2 = parent_pair_tuple  # Unpack the tuple
        for parent in [parent1, parent2]:  # Loop through the unpacked parent IDs
            if parent in selected_dna_branch_ids:
                filtered_parent_pairs[generation] = f"{parent1}&{parent2}"  # Reconstruct the parent_pair string
                break

    # Sort the filtered_parent_pairs by the generation numbers (keys)
    sorted_parent_pairs = sorted(filtered_parent_pairs.items())

    # Extract just the parent pairs, discarding the generation numbers
    sorted_parent_pairs_str_list = [pair for generation, pair in sorted_parent_pairs]

    # Concatenate into the final string using '|' as separator
    filtered_parent_pairs_str = '|'.join(sorted_parent_pairs_str_list)

#    print(f"390-Debug - individual_id: {individual_id}")
#    print(f"391-Debug - distant_ancestors_paths: {distant_ancestors_paths}")
    print(f"392-Debug - winning_path_names: {winning_path_names}")


    print(f"393-Debug - filtered_parent_pairs before sorting: {filtered_parent_pairs}")

    print(f"Filtered Parent Pairs String for {individual_id}: {filtered_parent_pairs_str}")

    individual_data['Filtered Ancestral Line'] = filtered_parent_pairs_str

    return individual_data, filtered_parent_pairs_str  # Return both individual_data and the formatted ancestral line string


# Initialize your empty combined_df_rows list here, before the main loop
combined_df_rows = []

# Main Loop
for dataset in filtered_datasets:
    individual_id = dataset.get_gen_person()

    # Reset global variables for each new individual
    visited_pairs = set()
    generation_table = []

    # Call the refactored process_individual function
    individual_data, filtered_parent_pairs_str = process_individual(individual_id, gedcom_instance, records)

    cm = individual_data['cM']
    sort = individual_data['Sort']

    # No need for final_output, we'll use filtered_parent_pairs_str directly
    combined_df_rows.append([individual_id, sort, individual_name, cm, filtered_parent_pairs_str])
    # Removed the second call to process_individual



import pandas as pd

# Global variables
visited_pairs = set()

# Assuming gedcom_instance and path_scores variables are already defined
filtered_individuals = [(dataset.get_anchor_gen1(), dataset.get_gen_person()) for dataset in gedcom_instance.filter_pool]

combined_df_rows = []  # Initialize your empty combined_df_rows list

# Main Loop
for dataset in filtered_datasets:
    individual_id = dataset.get_gen_person()

    # Reset global variables for each new individual
    visited_pairs = set()
    generation_table = []

    # Call the refactored process_individual function
    individual_data = process_individual(individual_id, gedcom_instance, records)

    # Now, individual_data will contain 'cM', 'Sort', and 'Filtered Ancestral Line'
    # You can use this information for further processing or output.

    # Translate IDs to Names
    name_paths = []
    for path in distant_ancestors_paths:
        name_path = [extract_name(records.get(id, '')) for id in path]
        name_paths.append(name_path)

    formatted_names = [f"{name}" for name in winning_path_names]
    final_output = "~".join(formatted_names)

    individual_data, filtered_parent_pairs_str = process_individual(individual_id, gedcom_instance, records)
    cm = individual_data['cM']
    sort = individual_data['Sort']

    # Swapping the positions of 'individual_name' and 'sort' in the output
    combined_df_rows.append([individual_id, sort, individual_name, cm, filtered_parent_pairs_str])


# Define whether to use hotlinks or not
USE_HOTLINK = True  # Set this to False if you don't want to use hotlinks

def create_hotlink(row):
    if USE_HOTLINK:
        url_base = "https://yates.one-name.net/tng/verticalchart.php?personID="
        additional_params = "&tree=tree1&parentset=0&display=vertical&generations=15"
        person_id = row['ID#']
        hotlink_url = f'{url_base}{person_id}{additional_params}'
        return f'<a href="{hotlink_url}">{person_id}</a>'
    else:
        return row['ID#']

combined_df = pd.DataFrame(combined_df_rows, columns=['ID#', 'Match to', 'Name', 'cM', 'Yates DNA Ancestral Line'])  # Excluded 'Score' from columns

combined_df['LUN#'] = combined_df.apply(lambda row: create_hotlink(row), axis=1)

ordered_columns = ['ID#', 'Match to', 'Name', 'cM', 'LUN#', 'Yates DNA Ancestral Line']  # Removed 'Score' from ordered columns
combined_df = combined_df[ordered_columns]

combined_df.index = combined_df.index + 1

# Sort the DataFrame as you wanted
combined_df = combined_df.sort_values(by=['Match to', 'Yates DNA Ancestral Line'], ascending=[False, True])

print(combined_df)

combined_df.to_excel('/content/output.xlsx', index=False)



Automatically selecting the first GEDCOM file.
GEDCOM contained 51006 total records
Records tagged and filtered by NPFX: 405
Manual filter IDs loaded: 0
After manual filter, total records: 1
314-Debug - winning_path_ids: ['I50522', 'I50519', 'I50517', 'I50515', 'I50513', 'I44014', 'I21091', 'I21089', 'I18899', 'I11851', 'I11866', 'I9912', 'I9914', 'I11798', 'I24380', 'I24382', 'I24502', 'I47548', 'I47549']
314-Debug - winning_path_ids: ['I50522', 'I50519', 'I50517', 'I50515', 'I50513', 'I44014', 'I21091', 'I21089', 'I18899', 'I11851', 'I11866', 'I9912', 'I9914', 'I11798', 'I24380', 'I24382', 'I24502', 'I47548', 'I47549']
392-Debug - winning_path_names: ['FehrenbachAngelaDe', 'SwinsonRita', 'SwinsonPaulRoss', 'BennettHazelBen', 'RictorInez', 'YatesJuliaAnn', 'YatesWilliam', 'YatesSimeon', 'YatesWilliam', 'YatesJoshua', 'YatesSamuel', 'YatesGeorge', 'YatesGeorge', 'YatesJohn', 'YatesThomas', 'YatesFrancis', 'YatesThomas', 'YatesJohn', 'YatesRichard']
393-Debug - filtered_parent_pairs bef

In [12]:
!pip install pandas
!pip install python-gedcom
!pip install openpyxl